# Vulnerability Ireland 

Population density is an important indicator to consider in a vulnerability assessment. The population density can be derived for each of the 'Small Area' units of the 2016 Census.

This script gets the population in each 'Small Area' in 2016 and the area of the 'Small Area' (in km<sup>2</sup>). These are then used to calculate the population density. 

## Setup

Load the R libraries:

In [1]:
# load the libraries
library(tidyverse)
library(sf)
library(raster)
library(dplyr)
library(rgdal)

Warning message:
"package 'sf' was built under R version 3.6.3"Linking to GEOS 3.9.0, GDAL 3.2.1, PROJ 7.2.1
Loading required package: sp

Attaching package: 'raster'

The following object is masked from 'package:dplyr':

    select

The following object is masked from 'package:tidyr':

    extract

rgdal: version: 1.4-4, (SVN revision 833)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
 Path to GDAL shared files: C:/Users/james/anaconda3/envs/ISVEHI/Lib/R/library/rgdal/gdal
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: C:/Users/james/anaconda3/envs/ISVEHI/Lib/R/library/rgdal/proj
 Linking to sp version: 1.3-1 


Set a temporary and export directory:

In [2]:
#set a directory for export 
exportDirectory <- "C:/GitHub/Climate-Ireland/Vulnerability/1_InputData/1b_PopulationDensity"

## 2.0 Population Data

The population data is derived from the 2016 census which can be imported directly from the CSO website (this is the default) or using a local version:

In [3]:
#get the data from the CSO website
smallAreaCSOData <- read.csv("https://www.cso.ie/en/media/csoie/census/census2016/census2016boundaryfiles/SAPS2016_SA2017.csv",  header=TRUE, sep=",")

#get the data locally
# smallAreaCSOData <- read.csv("../../CSOData/SAPS2016_SA2017.csv", header=TRUE, sep=",", stringsAsFactors = FALSE)

#get unique ID and population data only
smallAreaPopulation <- smallAreaCSOData[, c('GUID', 'T1_1AGETT'), drop = FALSE]
names(smallAreaPopulation)[2] <- 'populationTotal'

# head(smallAreaPopulation)

## 3.0 'Small Areas' Spatial Data

The spatial data for the Small Areas used within the 2016 census can be imported directly from the CSO website (this is the default) or using a local version. The area of each Small Area has already been calculated and is available in the 'Shape_Are' attribute of the data - this data is converted from m<sup>2</sup> to km<sup>2</sup>. 

In [14]:
#download the small area shapefile
download.file("http://data-osi.opendata.arcgis.com/datasets/c85e610da1464178a2cd84a88020c8e2_3.zip", 
              destfile = "./downloads/SmallAreasShp.zip" , mode='wb')
unzip("./downloads/SmallAreasShp.zip", exdir = "./downloads")

#read the data - Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015
smallAreaShp <- st_read('./downloads/d9876b9b-96eb-4f04-afec-300c5d0a276c2020328-1-wq0rek.ua66c.shp')

#use a local copy
# smallAreaShp <- st_read("D:/Spatial_Data/CSO/Census_2016_Small_Areas/Shapefiles/Extracted/Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015/Small_Areas_Ungeneralised__OSi_National_Statistical_Boundaries__2015.shp")

#convert to 'SpatialPolygonsDataFrame'
smallAreaShp <- as(smallAreaShp, "Spatial")
# head(smallAreaShp, n = 5)

#convert to irenet95 / irish transverse mercator
projIreNet95 <- "+proj=tmerc +lat_0=53.5 +lon_0=-8 +k=0.99982 +x_0=600000 +y_0=750000 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
smallAreaShp <- st_transform(smallAreaShp, projIreNet95)

#convert the area from m2 to km2
smallAreaShp$Shape__Are <- smallAreaShp$Shape__Are/1000000

#get the the unique IDs data and the shape areas only
smallAreaShp <- smallAreaShp[, c('GUID', 'SMALL_AREA', 'Shape__Are'), drop = FALSE]


## 4.0 Combining the data

The population data needs to be joined with the area data in order to calculate the density. This is done by matching the commmon filed of the GUID to merge to two datasets together.

In [15]:
smallAreaShpPop <-  merge(smallAreaShp, smallAreaPopulation, 'GUID')
# head(smallAreaShpPop)

## 5.0 Population Density

The population density can then be calculated and stored as a new field within the dataset. The final data is then exported to a csv for use in later analysis.

In [18]:
#Population Density Calculation
smallAreaShpPop$popDensity <- smallAreaShpPop$populationTotal/smallAreaShpPop$Shape__Are
head(smallAreaShpPop)

ERROR while rich displaying an object: Error in loadNamespace(name): there is no package called 'geojsonio'

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime]](obj)
8. repr_geojson.sf(o

GUID,SMALL_AREA,Shape__Are,populationTotal,geometry,popDensity
4c07d11d-f3af-851d-e053-ca3ca8c0ca7f,237103001,20.687235,272,MULTIPOLYGON (((621889.3 74...,13.14820
4c07d11d-f3b0-851d-e053-ca3ca8c0ca7f,237061001,16.337795,362,MULTIPOLYGON (((621889.3 74...,22.15721
4c07d11d-f3b1-851d-e053-ca3ca8c0ca7f,237025001,15.169654,236,MULTIPOLYGON (((628126.9 73...,15.55738
4c07d11d-f3b2-851d-e053-ca3ca8c0ca7f,237025003,6.722791,228,MULTIPOLYGON (((633200 7418...,33.91449
4c07d11d-f3b3-851d-e053-ca3ca8c0ca7f,107035001,19.751091,332,MULTIPOLYGON (((632245.4 68...,16.80920
4c07d11d-f3b4-851d-e053-ca3ca8c0ca7f,107041002,12.100627,215,MULTIPOLYGON (((635589 6868...,17.76767


In [19]:
#Scaled Population Density
smallAreaShpPop$popDensityZ <- scale(smallAreaShpPop$popDensity)

### 5.1 Export

In [20]:
#build export path
exportPath <- paste(exportDirectory, 'populationDensitySmallArea2016', sep = '/')
exportPath <- paste(exportPath, '.csv', sep = '')

write.csv(smallAreaShpPop, exportPath, row.names = FALSE)


**END**